In [1]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action="ignore",category=DeprecationWarning)
warnings.filterwarnings(action="ignore",category=FutureWarning)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import gc
import copy
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from datetime import datetime

from sklearn.metrics import roc_auc_score


from keras.layers import Dense, Input, Activation
from keras.layers import BatchNormalization,Add,Dropout
from keras.optimizers import Adam
from keras.models import Model, load_model
from keras import callbacks
from keras import backend as K

from time import time
import datetime

Using TensorFlow backend.


In [2]:
folder_path = '../data/IEEE-CIS-Fraud-Detection/'

In [3]:
df_train = pd.read_pickle(f'{folder_path}/df_train2.gzde', compression='gzip')#.iloc[:10000,:]
df_test = pd.read_pickle(f'{folder_path}/df_test2.gzde', compression='gzip')#.iloc[:10000,:]

In [4]:
columns = df_train.columns.tolist()
columns.remove('TransactionID')
columns.remove('TransactionDT')
columns.remove('isFraud')

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib
from tensorflow.python.framework import graph_util
import os

class VariationalAE(object):

    """ Variation Autoencoder (VAE) with an sklearn-like interface implemented using TensorFlow.

    This implementation uses probabilistic encoders and decoders using Gaussian
    distributions and  realized by multi-layer perceptrons. The VAE can be learned
    end-to-end.

    See "Auto-Encoding Variational Bayes" by Kingma and Welling for more details.
    """

    def __init__(self, X_dim, latent_dim, encoder_hidden_layer_sizes, decoder_hidden_layer_sizes, 
                 hidden_layer_activation_function, output_activation_function, kl_coef, drop_out, learning_rate, 
                 batch_size, epochs, random_state=0, **kwargs):
        
        
        self.X_dim = X_dim
        self.latent_dim = latent_dim
        self.encoder_hidden_layer_sizes = encoder_hidden_layer_sizes
        self.decoder_hidden_layer_sizes = decoder_hidden_layer_sizes
        
        if hidden_layer_activation_function == 'relu' or hidden_layer_activation_function == 'RELU':
            self.hidden_layer_activation_function = tf.nn.relu
        elif hidden_layer_activation_function == 'sigmoid' or hidden_layer_activation_function == 'SIGMOID':
            self.hidden_layer_activation_function = tf.nn.sigmoid
        elif hidden_layer_activation_function == 'tanh' or hidden_layer_activation_function == 'TANH':
            self.hidden_layer_activation_function = tf.nn.tanh
        else:
            raise Exception('unknow activation function name')
        
            
        if output_activation_function == 'relu' or output_activation_function == 'RELU':
            self.output_activation_function = tf.nn.relu
        elif output_activation_function == 'sigmoid' or output_activation_function == 'SIGMOID':
            self.output_activation_function = tf.nn.sigmoid
        elif output_activation_function == 'tanh' or output_activation_function == 'TANH':
            self.output_activation_function = tf.nn.tanh
        elif output_activation_function == 'softplus' or output_activation_function == 'SOFTPLUS':
            self.output_activation_function = tf.nn.softplus
        elif output_activation_function == 'softmax' or output_activation_function == 'SOFTMAX':
            self.output_activation_function = tf.nn.softmax
        else:
            raise Exception('unknow activation function name')
            
        self.kl_coef = kl_coef
        self.drop_out = drop_out
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs
        self.random_state = random_state
        self.is_training = False
        
        tf.set_random_seed(random_state)
        tf.reset_default_graph()
        self.build_graph()
        self.compile_graph()
        self.sess = tf.InteractiveSession()
        self.sess.run([tf.local_variables_initializer(), tf.global_variables_initializer(),])
        
        return


    def fit(self, X, X_val=None, **kwargs):
        
        his = []
        # Training
        n_sample = X.shape[0]
        n_steps = int(self.epochs*n_sample/self.batch_size)
        # input_queue = tf.train.slice_input_producer([X], shuffle=False, num_epochs=self.epochs)
        batch_X = tf.train.batch([X], batch_size=self.batch_size, enqueue_many=True)

        # self.sess.run([
        #     tf.local_variables_initializer(),
        #     tf.global_variables_initializer(),
        # ])
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=self.sess, coord=coord)

        try:
            # in most cases coord.should_stop() will return True
            # when there are no more samples to read
            # if num_epochs=0 then it will run for ever
            index = 1
            while not coord.should_stop():
                # will start reading, working data from input queue
                # and "fetch" the results of the computation graph
                # into raw_images and raw_labels
                batch_x = self.sess.run(batch_X)
                self.is_training = True
                self.sess.run((self.train_op), feed_dict={self.x: batch_x, self.keep_prob: 1 - self.drop_out})
                if (index % int(n_sample / self.batch_size) == 0):
                    d_ = {}
                    self.is_training = False
                    mse, loss, kle, reconstructione = self.sess.run((self.mse, self.loss, self.kle, self.reconstructione),feed_dict={self.x: X, self.keep_prob: 1.0})
                    d_ = {'loss': loss,'kle': kle, 'reconstructione': reconstructione, 'mse': mse}
                    if type(X_val)!=type(None):
                        val_mse, val_loss, val_kle, val_reconstructione = self.sess.run((self.mse, self.loss, self.kle, self.reconstructione),feed_dict={self.x: X_val, self.keep_prob: 1.0})
                        d_ = {'val_loss': val_loss,'val_kle': val_kle, 'val_reconstructione': val_reconstructione, 'val_mse': val_mse, **d_}
                    his.append(d_)    
                    
                index = index + 1
                if index == n_steps+1:
                    coord.request_stop()
        finally:
            coord.request_stop()
            coord.join(threads)

        return his

    def scores(self, X, **kwargs):
        his = []
        for _x in X:
            self.is_training = False
            mse, loss, kle, reconstructione = self.sess.run(
                (self.mse, self.loss, self.kle, self.reconstructione),
                feed_dict={self.x: [_x], self.keep_prob: 1.0})
            # print('epoch loss', loss)
            his.append(
                {'loss': loss,
                 'kle': kle,
                 'reconstructione': reconstructione,
                 'mse': mse})
        return his
    
        # latent
    def transform(self, X):
        latent_list = []
        for _x in X:
            latent_list.append(self.z.eval(session=self.sess, feed_dict={self.x: [_x], self.keep_prob: 1.0}))
        latent_array = np.array(latent_list)
        return latent_array

    # out put
    def predict(self, X):
        output_list = []
        for _x in X:
            output_list.append(self.y.eval(session=self.sess, feed_dict={self.x: [_x], self.keep_prob: 1.0}))
        output_array = np.array(output_list)
        return output_array

    # Gaussian MLP as encoder
    def _build_encoder(self):
        # initializers
        w_init = tf.contrib.layers.variance_scaling_initializer()
        b_init = tf.constant_initializer(0.)

        x_ = self.x
        input_dim = self.X_dim
        for index, unit in enumerate(self.encoder_hidden_layer_sizes):
            # 1st hidden layer
            w_variable_name_ = 'encoder_w{:d}'.format(index)
            b_variable_name_ = 'encoder_b{:d}'.format(index)
            w = tf.get_variable(w_variable_name_, [input_dim, unit], initializer=w_init)
            b = tf.get_variable(b_variable_name_, [unit], initializer=b_init)
            x_ = tf.matmul(x_, w) + b
            x_ = tf.layers.batch_normalization(x_, training=self.is_training)
            x_ = self.hidden_layer_activation_function(x_)
            x_ = tf.nn.dropout(x_, self.keep_prob)
            input_dim = unit

        # output layer
        wo = tf.get_variable('encoder_wo',[unit, self.latent_dim * 2], initializer=w_init)
        bo = tf.get_variable('encoder_bo',[self.latent_dim * 2], initializer=b_init)
        gaussian_params = tf.matmul(x_, wo) + bo
        gaussian_params = tf.layers.batch_normalization(gaussian_params, training=self.is_training)

        # The mean parameter is unconstrained
        self.mu = gaussian_params[:, :self.latent_dim]
        # The standard deviation must be positive. Parametrize with a softplus and
        # add a small epsilon for numerical stability
        self.sigma = 1e-6 + tf.nn.softplus(gaussian_params[:, self.latent_dim:])

        # sampling by re-parameterization technique
        eps = tf.random_normal(tf.shape(self.mu), mean=0, stddev=1)
        self.z = self.mu + self.sigma * eps


    # Bernoulli MLP as decoder
    def _build_decoder(self):

        # initializers
        w_init = tf.contrib.layers.variance_scaling_initializer()
        b_init = tf.constant_initializer(0.)

        x_ = self.z
        input_dim = self.latent_dim
        for index, unit in enumerate(self.decoder_hidden_layer_sizes):
            # 1st hidden layer
            w_variable_name_ = 'decoder_w{:d}'.format(index)
            b_variable_name_ = 'decoder_b{:d}'.format(index)
            w = tf.get_variable(w_variable_name_, [input_dim, unit], initializer=w_init)
            b = tf.get_variable(b_variable_name_, [unit], initializer=b_init)
            x_ = tf.matmul(x_, w) + b
            x_ = tf.layers.batch_normalization(x_, training=self.is_training)
            x_ = self.hidden_layer_activation_function(x_)
            x_ = tf.nn.dropout(x_, self.keep_prob)
            input_dim = unit

        # output layer-mean
        wo = tf.get_variable('decoder_wo',[unit, self.X_dim], initializer=w_init)
        bo = tf.get_variable('decoder_bo',[self.X_dim], initializer=b_init)
        params = tf.matmul(x_, wo) + bo
        params = tf.layers.batch_normalization(params, training=self.is_training)
        self.y = self.output_activation_function(params, name='output')

        return

    def _loss(self):

        local_epsilon = 1e-6
        # marginal_likelihood = self.x * tf.log(tf.clip_by_value(self.y, local_epsilon, 1.0)) + \
        #                             (1 - self.x) * tf.log(tf.clip_by_value(1 - self.y, local_epsilon, 1.0))
        y_ = tf.clip_by_value(self.y, local_epsilon, 1 - local_epsilon)
        x_ = tf.clip_by_value(self.x, local_epsilon, 1 - local_epsilon)
        marginal_likelihood = x_ * tf.log(y_) + (1 - x_) * tf.log(1 - y_)
        marginal_likelihood = -1 * tf.reduce_sum(marginal_likelihood, 1)

        mu_ = tf.clip_by_value(self.mu, local_epsilon, 1 - local_epsilon)
        sigma_ = tf.clip_by_value(self.sigma, local_epsilon, 1 - local_epsilon)
        kl_divergence = 1 + tf.log(tf.square(sigma_)) - tf.square(mu_) - tf.square(sigma_)
        kl_divergence = -self.kl_coef * tf.reduce_sum(kl_divergence, 1)

        self.loss = tf.reduce_mean(marginal_likelihood + kl_divergence, name='loss')
        self.mse = tf.sqrt(tf.losses.mean_squared_error(self.y, self.x), name='mse')
        self.kle = tf.reduce_mean(kl_divergence, name='kle')
        self.reconstructione = tf.reduce_mean(marginal_likelihood, name='reconstructione')
        return


    def compile_graph(self):
        
        # loss
        self._loss()
        # train
        self.train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        return

    def build_graph(self):

        self.keep_prob = tf.placeholder(tf.float32, name="keep_prob")
        self.x = tf.placeholder(tf.float32, [None, self.X_dim], name='input')
        # encoding
        self._build_encoder()
        # decoding
        self._build_decoder()
        return


In [5]:
def process(folds, df_train,df_test, columns, verbose=0):

#     aucs = list()
    his = []
    training_start_time = time()
    df_valid_pred = pd.DataFrame()
    df_test_pred = pd.DataFrame()
    if type(df_test) != type(None):
        df_test_pred['TransactionID'] = df_test['TransactionID'].values
    
    scaler = MinMaxScaler()
    scaler.fit(df_train[columns].values)
        
    for fold, (trn_idx, test_idx) in enumerate(folds.split(df_train)):
        start_time = time()
        if verbose > 1:
            print('Training on fold {}'.format(fold + 1))
        
        vae = VariationalAE(**params)
        train_his = vae.fit(scaler.transform(df_train.iloc[trn_idx][df_train['isFraud']==0][columns].values))
        scores = vae.scores(scaler.transform(df_train.iloc[test_idx][columns].values))
        
        df_valid_pred_i = pd.DataFrame(scores)
        df_valid_pred_i['TransactionID'] = df_train['TransactionID'].values[test_idx]
        df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
        
        y_test_pred = None
        if type(df_test)!=type(None):
            scores = vae.scores(scaler.transform(df_test[columns].values))
            df_test_pred_i = pd.DataFrame(scores)
            df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=1)
        
        
#         aucs.append(clf.best_score['valid_1']['auc'])
        his.append({'train_his':train_his, 'df_valid_pred_i':df_valid_pred_i, 'test_idx':test_idx})
        if verbose > 0:
            print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))
    his = pd.DataFrame(his)
            
    df_valid_pred = df_valid_pred.sort_values(by=['TransactionID'])
    df_valid_pred = df_valid_pred.reset_index(drop=True)

    if type(df_test) != type(None):
        df_test_pred = df_test_pred.sort_values(by=['TransactionID'])
        df_test_pred = df_test_pred.reset_index(drop=True)
        
        df_test_pred_mean = pd.DataFrame()
        df_test_pred_mean['TransactionID'] = df_test_pred['TransactionID'].values
        df_test_pred_mean['loss'] = df_test_pred['loss'].mean(axis=1).values
        df_test_pred_mean['kle'] = df_test_pred['kle'].mean(axis=1).values
        df_test_pred_mean['reconstructione'] = df_test_pred['reconstructione'].mean(axis=1).values
        df_test_pred_mean['mse'] = df_test_pred['mse'].mean(axis=1).values
    
    if verbose > 0:
        print('-' * 30)
        print('Training has finished.')
        print('Total training time is {}'.format(str(datetime.timedelta(seconds=time() - training_start_time))))
        print('-' * 30)
    return his, df_valid_pred, df_test_pred_mean

In [6]:
# df_train2 = pd.read_pickle(f'{folder_path}/df_train2.gzde', compression='gzip').iloc[10000:20000,:]

In [7]:
sorted_columns = ['PCA_C_1',
 'C1_fq_enc',
 'C13',
 'PCA_C_2',
 'C1',
 'C14',
 'D2',
 'V70',
 'M4_target_mean',
 'uid3_TransactionAmt_mean',
 'addr1_fq_enc',
 'card2_TransactionAmt_mean',
 'addr1__card1',
 'V258',
 'M5',
 'uid3_TransactionAmt_std',
 'uid3_fq_enc',
 'D15_to_std_addr1',
 'D15_to_mean_addr1',
 'C11_fq_enc',
 'card1',
 'C11',
 'card1_TransactionAmt_mean',
 'D2_fq_enc',
 'uid2_TransactionAmt_mean',
 'P_emaildomain__C2',
 'M6',
 'card2_fq_enc',
 'D1',
 'C13_fq_enc',
 'V317',
 'C2',
 'TransactionAmt',
 'addr1',
 'uid2_fq_enc',
 'V294',
 'uid_TransactionAmt_mean',
 'C14_fq_enc',
 'card1_fq_enc',
 'C8',
 'TransactionAmt_log1p',
 'card1_count_full',
 'card3_TransactionAmt_mean',
 'Card_ID',
 'M4',
 'C2_fq_enc',
 'P_emaildomain_fq_enc',
 'ProductCD_target_mean',
 'uid2_TransactionAmt_std',
 'uid_fq_enc',
 'card5__P_emaildomain',
 'card6',
 'uid_TransactionAmt_std',
 'card1_TransactionAmt_std',
 'id_20',
 'card2_count_full',
 'C6_fq_enc',
 'card2_TransactionAmt_std',
 'V87',
 'card2__dist1',
 'card2',
 'C6',
 'R_emaildomain',
 'PCA_C_0',
 'C5',
 'TransactionAmt_Log',
 'TransactionAmt_decimal',
 'D15_to_std_card1',
 'D3',
 'PCA_V_28',
 'uid',
 'V91',
 'V45',
 'uid3',
 'card5_TransactionAmt_mean',
 'card2__id_20',
 'D3_fq_enc',
 'V69',
 'card6_count_full',
 'C5_fq_enc',
 'V283',
 'uid2',
 'V257',
 'DeviceInfo_version',
 'TransactionAmt_to_mean_card1',
 'C12',
 'card1__card5',
 'V189',
 'P_emaildomain',
 'D1_fq_enc',
 'D15_to_mean_card1',
 'TransactionAmt_to_mean_card_id',
 'card3_TransactionAmt_std',
 'R_emaildomain_prefix',
 'C8_fq_enc',
 'ProductCD',
 'P_emaildomain_prefix',
 'id_19',
 'V310',
 'id_17',
 'M3',
 'C10',
 'V245',
 'V62',
 'id_31_device',
 'V187',
 'card3',
 'DeviceInfo_device_fq_enc',
 'D14',
 'V243',
 'V53',
 'V90',
 'V67',
 'C9_fq_enc',
 '_Days',
 'email_check',
 'V312',
 'mean_last',
 'V313',
 'C9',
 'id_05',
 'id_09',
 'V156',
 'P_emaildomain_1',
 'V54',
 'V133',
 'DeviceType',
 'V308',
 'C12_fq_enc',
 'V30',
 'V261',
 'P_emaildomain_2',
 'V274',
 'card5_fq_enc',
 'V307',
 'V83',
 'V149',
 'V61',
 'V201',
 'id_30_device',
 'V102',
 'card5',
 'id_02_to_std_addr1',
 'id_30_fq_enc',
 'V82',
 'V13',
 'D8_fq_enc',
 'first_value_addr1',
 'DT_day',
 'V86',
 'DeviceInfo__P_emaildomain',
 'V320',
 'V76',
 'D5',
 'V35',
 'max_last',
 'addr2',
 'V188',
 'V264',
 'V259',
 'C10_fq_enc',
 'TransactionAmt_to_std_card_id',
 'PCA_D_1',
 'V75',
 'V315',
 'C4_fq_enc',
 'device_version',
 'DeviceInfo_version_fq_enc',
 'V56',
 'id_38',
 'V281',
 'first_value_card1',
 'id_31_device_fq_enc',
 'V55',
 'V44',
 'DT_hour',
 'M9',
 'card4_count_full',
 'min_last',
 'DeviceInfo_device',
 'id_18',
 'C4',
 'V12',
 'dist1_fq_enc',
 'card5_TransactionAmt_std',
 '_Second',
 'M_sum',
 'id_37',
 'V29',
 'DeviceInfo_fq_enc',
 'D11',
 'V262',
 'V198',
 'V314',
 'V256',
 'id_02_to_mean_addr1',
 'V38',
 'R_emaildomain_2',
 '_Hours',
 'V66',
 'D8',
 'V296',
 'V279',
 'V78',
 'D5_fq_enc',
 'V147',
 'V306',
 'V20',
 '_Minutes',
 'V49',
 'V48',
 'id_30_version_fq_enc',
 'V275',
 'V335',
 'V280',
 'V127',
 'V36',
 'R_emaildomain_1',
 'V19',
 'card5_count_full',
 'id_28',
 'screen_height',
 'id_14',
 'id_33',
 'V23',
 'V244',
 'V287',
 'PCA_D_2',
 'id_02__id_20',
 'V326',
 'id_33_fq_enc',
 'id_32',
 'addr2_fq_enc',
 'V24',
 'id_30',
 'dist1',
 'V277',
 'C7',
 'V152',
 'V232',
 'M7',
 'std_last',
 'V282',
 'device_name',
 'V199',
 'card4',
 'V131',
 'V99',
 'V324',
 'V139',
 'id_02__D8',
 'V253',
 'id_33_1',
 'id_02',
 'PCA_D_0',
 'V319',
 'V160',
 'V155',
 'V336',
 'M1',
 'V251',
 'V50',
 'V108',
 'M_na',
 'screen_width',
 'V125',
 'V57',
 'id_03',
 'V239',
 'V115',
 'V123',
 'count_cluster',
 'V109',
 'V298',
 'V231',
 'PCA_D_6',
 'V291',
 'V215',
 'V148',
 'V124',
 'V293',
 'card3_count_full',
 'Transaction_hour',
 'V162',
 'version_id_30',
 'V304',
 'V16',
 'V309',
 'V134',
 'V273',
 'V105',
 'V219',
 'C3_fq_enc',
 'V31',
 'V272',
 'V289',
 'V58',
 'V217',
 'V255',
 'V271',
 'V216',
 'V327',
 'V126',
 'V128',
 'V165',
 'card3_fq_enc',
 'V197',
 'V132',
 'id_30_version',
 'V34',
 'V143',
 'id_34',
 'V332',
 'V323',
 'V25',
 'V161',
 'V200',
 'V210',
 'V5',
 'V318',
 'browser_id_31',
 'id_29',
 'V11',
 'V248',
 'V242',
 'OS_id_30',
 'V237',
 'C3',
 'V65',
 'DeviceInfo',
 'V292',
 'V15',
 'V229',
 'V183',
 'V190',
 'V302',
 'V7',
 'PCA_D_5',
 'V96',
 'V101',
 'V135',
 'V301',
 'V9',
 'V158',
 'V153',
 'V46',
 'V14',
 'V193',
 'V116',
 'V252',
 'id_02_to_mean_card1',
 'id_30_device_fq_enc',
 'PCA_D_7',
 'V112',
 'V94',
 'V316',
 'V150',
 'V311',
 'D7',
 'V159',
 'V8',
 'V180',
 'V142',
 'V321',
 'V195',
 'V325',
 '_Weekdays',
 'make_day_feature',
 'V186',
 'V227',
 'id_25',
 'V100',
 'V1',
 'V3',
 'V6',
 'V322',
 'V331',
 'V303',
 'V221',
 'V26',
 'V2',
 'V74',
 'V172',
 'V286',
 'V130',
 'V10',
 'V334',
 'V207',
 'V122',
 'V299',
 'V157',
 'V337',
 'V226',
 'V167',
 'V154',
 'V95',
 'V79',
 'V290',
 'id_33_0',
 'M2',
 'clusters_D',
 'V51',
 'V284',
 'V151',
 'V98',
 'V297',
 'V191',
 'V144',
 'V119',
 'V269',
 'V27',
 'V141',
 'V192',
 'V138',
 'V120',
 'V110',
 'clusters_C',
 'had_id',
 'V111',
 'V118',
 'V196',
 'V113',
 'V114',
 'V121',
 'V28',
 'V117',
 'V278',
 'V249',
 'V305',
 'V240',
 'id_36',
 'id_35',
 'V241',
 'V41',
 'V328',
 'V247',
 'id_27',
 'id_23',
 'id_22',
 'id_08',
 'V88',
 'V250',
 'V107',
 'V204',
 'V214',
 'V136',
 'V209',
 'V137',
 'V330',
 'V89',
 'V140',
 'V145',
 'V254',
 'id_24',
 'V270',
 'PCA_D_3',
 'Transaction_day_of_week',
 'V178',
 'V33',
 'V104',
 'V146',
 'V68',
 'V163',
 'id_36_count_full',
 'V73',
 'V213',
 'V168',
 'V236',
 'V173',
 'V97',
 'V106',
 'V212',
 'V129',
 'V263',
 'V37',
 'V164',
 'V230',
 'id_26',
 'V174',
 'M8',
 'V166',
 'V184',
 'V233',
 'V175',
 'D12',
 'V300',
 'V235',
 'V234',
 'V288',
 'V205',
 'V238',
 'V329',
 'V246',
 'V194',
 'V47',
 'DT_day_week',
 'V218',
 'make_hour_feature',
 'id_16',
 'V211',
 'id_15',
 'D6',
 'V32',
 'V222',
 'V182',
 'V228',
 'PCA_D_4',
 'V177',
 'V339',
 'id_21',
 'V267',
 'V4',
 'V185',
 'id_07',
 'V179',
 'D7_fq_enc',
 'V338',
 'V103',
 'V206',
 'V265',
 'id_02_to_mean_card4',
 'V260',
 'id_12',
 'V333',
 'id_10',
 'id_02_to_std_card4',
 'V181',
 'dist2',
 'V52',
 'dist2_fq_enc',
 'C7_fq_enc',
 'V202',
 'V276',
 'V77',
 'V208',
 'D9',
 'V176',
 'V171',
 'V224',
 'id_11',
 'D13',
 'id_04',
 'V203',
 'V225',
 'V170',
 'TransactionAmt_check',
 'V295',
 'V220',
 'V169',
 'R_emaildomain_fq_enc',
 'V268',
 'D6_fq_enc',
 'id_06',
 'V285',
 'V266',
 'V223',
 'id_01']

In [10]:
folds = KFold(n_splits=8, shuffle=False, random_state=42)
params = {
    'X_dim': len(sorted_columns[:200]),
    'latent_dim': 2,
    'encoder_hidden_layer_sizes':[1024, 256],
    'decoder_hidden_layer_sizes':[64],
    'hidden_layer_activation_function':'relu',
    'output_activation_function':'sigmoid',
    'kl_coef': .5,
    'drop_out': 0.1,                
    'learning_rate': 0.001,
    'batch_size': 16,
    'epochs': 200,
}

In [ ]:
his, df_valid_pred, df_test_pred = process(folds, df_train,df_test, sorted_columns[:200], verbose=0)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


In [ ]:
his

In [ ]:
123

In [52]:
df_valid_pred

,loss,kle,reconstructione,mse,TransactionID
0,114.867157,0.502394,114.364761,0.152810,2987000
1,99.665977,0.519566,99.146408,0.121638,2987001
2,106.663673,0.502882,106.160789,0.100766,2987002
3,127.823006,0.658024,127.164978,0.146039,2987003
4,166.206467,0.031082,166.175385,0.143271,2987004
...,...,...,...,...,...
9995,161.074203,0.534761,160.539444,0.134931,2996995
9996,120.351067,1.159901,119.191162,0.186028,2996996
9997,108.676117,1.109847,107.566269,0.115891,2996997
9998,111.611336,0.504733,111.106606,0.135386,2996998


In [53]:
df_test_pred

,TransactionID,loss,kle,reconstructione,mse
0,2997000,118.942210,0.770884,118.171326,0.146101
1,2997001,111.266341,0.517203,110.749136,0.135822
2,2997002,110.686563,0.491461,110.195102,0.120008
3,2997003,98.938614,0.362082,98.576534,0.102681
4,2997004,111.863249,0.955579,110.907671,0.151147
...,...,...,...,...,...
9995,3006995,198.098608,2.875278,195.223331,0.200928
9996,3006996,122.023468,0.497102,121.526366,0.203341
9997,3006997,120.004228,0.463054,119.541174,0.198779
9998,3006998,128.002359,0.535765,127.466599,0.212015
